1. 根据提供的kaggle JD评论语料进行文本分类训练
https://www.kaggle.com/datasets/dosonleung/jd_comment_with_label
2. 调整模型训练参数，添加tensorboard跟踪，对比bert冻结和不冻结之间的训练差异。
3. 保存模型进行分类预测。

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

import pandas as pd
from tqdm import tqdm

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter()

In [8]:
df = pd.read_excel("jd_comment_data.xlsx")
df.head()

,爬取时间(__time),爬取链接(__url),商品ID(product_id),评价时间(publish_time),评分（总分5分）(score),评价内容(content),评价者(author_name),评价者会员等级(author_level),商品sku(product_sku),评价标签(tags)
0,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550631798,5,此用户未填写评价内容,j***e,注册会员,单机版 小D黑色,[]
1,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633151,5,此用户未填写评价内容,c***n,钻石会员,单机版 小D黑色,[]
2,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633330,3,此用户未填写评价内容,j***1,银牌会员,单机版 小D黑色,[]
3,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633401,5,此用户未填写评价内容,苗***4,钻石会员,单机版 小D黑色,[]
4,2019-03-08 00:50:33,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633461,5,此用户未填写评价内容,J***3,注册会员,单机版 小D黑色,[]


In [9]:
df.columns

Index(['爬取时间(__time)', '爬取链接(__url)', '商品ID(product_id)', '评价时间(publish_time)',
       '评分（总分5分）(score)', '评价内容(content)', '评价者(author_name)',
       '评价者会员等级(author_level)', '商品sku(product_sku)', '评价标签(tags)'],
      dtype='object')

In [11]:
filterd= df['评价内容(content)'] != "此用户未填写评价内容"
data_df = df[filterd][['评价内容(content)','评分（总分5分）(score)']]

In [12]:
data_df.head()

,评价内容(content),评分（总分5分）(score)
15,一般般，一分钱一分货吧,1
18,商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。,4
19,。。。,5
22,刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局,5
25,还好还好还好还好红红火火好很好好,5


In [14]:
data = data_df.values
data

array([['一般般，一分钱一分货吧', 1],
       ['商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。', 4],
       ['。。。', 5],
       ...,
       ['没有色差，穿上很舒服，到货快。', 5],
       ['可以', 5],
       ['物有所值 客服贴心东西收到以后马上查看，发现与图片描述一致，超级喜欢，卖家发货 速度很快 ，，服务也很到位，给老板点个赞，下次还会来购买.........',
        5]], dtype=object)

In [15]:
train, test = train_test_split(data)
print(train.shape, test.shape)

(33316, 2) (11106, 2)


In [16]:
# 分词器
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-bert-wwm")

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

g:\Hmsoft\Anaconda\package\envs\py312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liuleilei\.cache\huggingface\hub\models--hfl--chinese-bert-wwm. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [17]:
# # 自定义DataLoader创建方法
def wrap_data(batch_data):
    comments, lables = [], []

    for data in batch_data:
        comments.append(data[0])
        lables.append(data[1])

    input_data = tokenizer(
        comments, return_tensors="pt", padding=True, truncation=True, max_length=512
    )
    lables = torch.tensor(lables)

    return input_data, lables


train_dl = DataLoader(
    train,
    batch_size=20,
    shuffle=True,
    collate_fn=wrap_data,
)

test_dl = DataLoader(
    test,
    batch_size=20,
    shuffle=False,
    collate_fn=wrap_data,
)

In [21]:
# model_1 模型微调 Supervised Fine Tuning
# model_2 迁移学习 Transfer Learning 冻结bert
model_1 = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-bert-wwm", num_labels=5)
model_2 = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-bert-wwm", num_labels=5)
model_1.to(device)
model_2.to(device)
model_2.trainable = False  # 冻结模型参数

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(14559935 bytes read, 397018523 more expected)', IncompleteRead(14559935 bytes read, 397018523 more expected))

In [ ]:
# loss, optimizer
loss_fn1 = nn.CrossEntropyLoss()
optim1 = Adam(model_1.parameters(), lr=1e-5)
loss_fn2 = nn.CrossEntropyLoss()
optim2 = Adam(model_2.parameters(), lr=1e-5)


In [ ]:
model1_train_loss_cnt = 0

for epoch in range(10):
    pbar = tqdm(train_dl)
    for input_data, lables_data in pbar:
        datas = {k: v.to(device) for k, v in input_data.items()}
        lables = lables_data.to(device)

        result = model_1(**datas)
        loss = loss_fn1(result.logits, lables)

        pbar.set_description(f"epoch {epoch} loss: {loss.item():.4f}")
        writer.add_scalar("model1/train_loss", loss, model1_train_loss_cnt)
        model1_train_loss_cnt += 1

        loss.backward()
        optim1.step()
        optim1.zero_grad()
torch.save(model_1.state_dict(), "model_1.pt")

In [ ]:
model2_train_loss_cnt = 0
for epoch in range(10):
    pbar = tqdm(train_dl)
    for input_data, lables_data in pbar:
        datas = {k: v.to(device) for k, v in input_data.items()}
        lables = lables_data.to(device)

        result = model_2(**datas)
        loss = loss_fn2(result.logits, lables)

        pbar.set_description(f"epoch {epoch} loss: {loss.item():.4f}")
        writer.add_scalar("model2/train_loss", loss, model2_train_loss_cnt)
        model2_train_loss_cnt += 1

        loss.backward()
        optim2.step()
        optim2.zero_grad()
torch.save(model_2.state_dict(), "model_2.pt")

In [ ]:
model_1.eval()
model_2.eval()

correct1, correct2 = 0, 0

for input_data, lables_data in tqdm(test_dl):
    datas = {k: v.to(device) for k, v in input_data.items()}
    lables = lables_data.to(device)

    with torch.no_grad():
        result1 = model_1(**datas)
        result2 = model_2(**datas)

    pred1 = torch.argmax(result1.logits, dim=1)
    pred2 = torch.argmax(result2.logits, dim=1)

    correct1 += (pred1 == lables).sum()
    correct2 += (pred2 == lables).sum()

In [ ]:
model_1.load_state_dict()